In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.colors

In [2]:
# 示例数据加载
df = pd.read_excel("1219campaignsummary200days20241226175207.xlsx", skiprows=3)

# 将 DATE 列转换为 datetime 类型
df['DATE'] = pd.to_datetime(df['DATE'])

# 去除包含 NaN 值的行
df_cleaned = df.dropna(subset=['CAMPAIGN ID', 'PROMOTED SKU'])

# 将 CAMPAIGN ID 和 PROMOTED SKU 转换为不带小数点的字符串
df_cleaned['CAMPAIGN ID'] = df_cleaned['CAMPAIGN ID'].astype(str).apply(lambda x: x.split('.')[0])
df_cleaned['PROMOTED SKU'] = df_cleaned['PROMOTED SKU'].astype(str).apply(lambda x: x.split('.')[0])

df = df_cleaned.copy()

C:\Users\cs\AppData\Local\Temp\ipykernel_21624\1014889224.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['CAMPAIGN ID'] = df_cleaned['CAMPAIGN ID'].astype(str).apply(lambda x: x.split('.')[0])
C:\Users\cs\AppData\Local\Temp\ipykernel_21624\1014889224.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['PROMOTED SKU'] = df_cleaned['PROMOTED SKU'].astype(str).apply(lambda x: x.split('.')[0])


In [63]:
unique_primary_values = df['CAMPAIGN ID'].unique()
primary_label = 'CAMPAIGN ID'
secondary_label = 'PROMOTED SKU'
selected_primary = '286776'
selected_metric = 'TOTAL SALES'
date_range =  [df['DATE'].min().date(), df['DATE'].max().date()]
date_range = [pd.to_datetime(date) for date in date_range]
# 数据过滤
filtered_df = df[(df['DATE'].dt.date >= date_range[0].date()) &
                 (df['DATE'].dt.date <= date_range[1].date()) &
                 (df[primary_label] == selected_primary)]

# 初始关系图数据
grouped_df = filtered_df.groupby([primary_label, secondary_label], as_index=False).agg({
    selected_metric: 'sum'
})

related_secondary_values = grouped_df[secondary_label].unique()
reverse_filtered_df = df[df[secondary_label].isin(related_secondary_values)]
reverse_grouped_df = reverse_filtered_df.groupby([secondary_label, primary_label], as_index=False).agg({
    selected_metric: 'sum'
})

In [67]:
# 根据primary_label调整合并顺序
if primary_label == 'PROMOTED SKU':
    # 当 primary_label 为 'PROMOTED SKU' 时，反转顺序
    combined_data = pd.concat([
        grouped_df.rename(columns={"PROMOTED SKU": "Source", "CAMPAIGN ID": "Target", "TOTAL SALES": "Value"}),
        reverse_grouped_df.rename(columns={"CAMPAIGN ID": "Source", "PROMOTED SKU": "Target", "TOTAL SALES": "Value"})
    ])
else:
    # 默认顺序
    combined_data = pd.concat([
        grouped_df.rename(columns={"CAMPAIGN ID": "Source", "PROMOTED SKU": "Target", "TOTAL SALES": "Value"}),
        reverse_grouped_df.rename(columns={"PROMOTED SKU": "Source", "CAMPAIGN ID": "Target", "TOTAL SALES": "Value"})
    ])

# 替换 TOTAL SALES 为 0 的值为一个小值，避免 Sankey 图显示异常
combined_data["Value"] = combined_data["Value"].replace(0, 1e-6)

# 定义颜色列表
color_palette = plotly.colors.qualitative.Set3
combined_data["Color"] = [
    "grey" if value == 1e-6 else color_palette[i % len(color_palette)]
    for i, value in enumerate(combined_data["Value"])
]
# 创建 Sankey 图所需的唯一标签
unique_labels = pd.concat([combined_data["Source"], combined_data["Target"]]).unique()
label_to_index = {label: idx for idx, label in enumerate(unique_labels)}

# 映射标签到索引
combined_data["Source"] = combined_data["Source"].map(label_to_index)
combined_data["Target"] = combined_data["Target"].map(label_to_index)

# 创建 Sankey 图
fig = go.Figure(data=[go.Sankey(
    textfont=dict(size=12, color='black'),
    node=dict(
        pad=30,
        thickness=20,
        line=dict(color="yellow", width=0.5),
        label=[str(label) for label in unique_labels],
        color='blue'
    ),
    link=dict(
        source=combined_data["Source"],
        target=combined_data["Target"],
        value=combined_data["Value"],
        color=combined_data["Color"],
        hovertemplate="Source: %{source.label}<br>Target: %{target.label}<br>Total Sales: %{value}<extra></extra>"
    )
)])

# 更新图表布局并显示
fig.update_layout(
    title_text="CAMPAIGN ID - PROMOTED SKU - CAMPAIGN ID 总销售额 Sankey 图",
    font_size=10,
    annotations=[
        dict(
            text="灰色表示销量为 0",
            x=0.5,
            y=-0.1,
            showarrow=False
        )
    ]
)
fig.show()